# Simulated Annealing search

We will show how we can use the class SimulatedAnnealing, which is in the Optimpy library. By convention we created every algorithm to find the solution where has gotten minimum value.

For this, it is necessary:

To overwrite the following functions
* get_neighbor (Required)
* update_temperature (Required)

And, to define
* the objective function $f$
* the constraints list

We will explain every step which is necessary to start to build the first search model.

## Imports

In [1]:
import sys
import os

library_path = "/home/dell/Documentos/Git_proejcts/optimizacion-con-metaheuristicas/"
sys.path.append(os.path.abspath(library_path))


In [2]:
from optimpy.heuristic.SimulatedAnnealing_search import SimulatedAnnealing
from optimpy.utils.helpers import *
from pprint import pprint
import numpy as np 
import copy 

## Travelling salesman problem


\begin{equation}
    \label{eq:TSP}
    \begin{array}{rll}
    \text{minimize:} & f(x) = d(x_n, x_1) + \sum_{i=1}^{n-1} d(x_i, x_{i+1}) &  \\
    \text{such that:} & x_i \in \{1,2,\cdots,n\} & \\
    \end{array}
\end{equation} 

where $d(x_i, x_j)$ is the distance from city $x_i$ to city $x_j$, $n$ is the number of cities and $x$ is a permutation of $n$ cities.

In [3]:
import random 
import math

num_cities = 10
dist_matrix = \
[\
[0,49,30,53,72,19,76,87,45,48],\
[49,0,19,38,32,31,75,69,61,25],\
[30,19,0,41,98,56,6,6,45,53],\
[53,38,41,0,52,29,46,90,23,98],\
[72,32,98,52,0,63,90,69,50,82],\
[19,31,56,29,63,0,60,88,41,95],\
[76,75,6,46,90,60,0,61,92,10],\
[87,69,6,90,69,88,61,0,82,73],\
[45,61,45,23,50,41,92,82,0,5],\
[48,25,53,98,82,95,10,73,5,0],\
]


### Objective function

In [4]:
@checkargs
def f_salesman(x : (list,np.ndarray)) -> float:
    global dist_matrix
    total_dist = dist_matrix[x[-1]][0]
    for i in range(1,len(x)):
        u,v = x[i], x[i-1]
        total_dist+= dist_matrix[u][v]

    return float(total_dist)

### Constraints 

In [5]:
@checkargs
def g_salesman(x : np.ndarray) -> bool:

    size = len(x)
    size_ = len(np.unique(x))
    return size == size_

constraints_list= [g_salesman]

### Initial solution
In this case, we create the initial solution using a greedy strategy.

In [6]:
def getInitialSolutionTS(distance_matrix, total_cities) -> np.ndarray:
    Solution = [0]
    remaining_cities  = list(range(1,total_cities))
    
    while len(remaining_cities) != 0:
        from_ =Solution[-1]
        to_ = remaining_cities[0]
        dist = distance_matrix[from_][to_]
        
        for i in range(1, len(remaining_cities)):
            distance = distance_matrix[from_][remaining_cities[i]]
            if distance < dist:
                to_ = remaining_cities[i]
                dist = distance
        Solution.append(to_)
        ind = remaining_cities.index(to_)
        remaining_cities.pop(ind)
        
    return np.array(Solution)

### Initialize Solver


#### Overwritting functions
**Note:** *It is necessary to implement get_neighbor. If not, the algorithm does not work.*
- **get_neighbor:** Return only one solution which is a *random* variation of current solution.

In [7]:
class TravellingSalesman_solver(SimulatedAnnealing):

    @checkargs
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
    @checkargs
    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        x_ = x.copy()
        N = len(x_)
        index1 = random.randint(1, N-1)
        index2 = random.randint(1, N-1)
    
        while index2 ==index1:
            index2 = random.randint(1, N-1)
    
        v = x[index1]
        x_ = list(x_[v != x_])
        x_ = x_[:index2] + [v] + x_[index2:]
        return np.array(x_)



Finally, we have finished the necessary steps for build our solver.
Now, we create a solver instance. We have to call optimize function, which finds the best solution with the following parameters:
- Init: Numpy array, represent the initial solution or function which generates a random initial solution (this solution should be a numpy array).
- IniTemperature:  Floating value, it define how much allow worse solutions.
- Epsilon: it means what's the final temperature.

In [8]:
TravellingSalesman = TravellingSalesman_solver( f_salesman, constraints_list)
init_path1 = getInitialSolutionTS(dist_matrix, num_cities)
print("Initialize search with this initial point {}".format(init_path1))


Initialize search with this initial point [0 5 3 8 9 6 2 7 1 4]


In [9]:
TravellingSalesman.optimize(init_path1, 10000.0 , 0.1)
print(TravellingSalesman)

Simulated Annealing: 
 f(X) = 269.0 
 X = [0 5 1 4 3 8 9 6 7 2] 
 


## Results
Now, we will check the algorithm's behavior, for this is necessary to evaluate the results for a specific number of runs. The Optimpy library has a function called **get_stats** (it is in **utils.helpers**), which returns a dictionary with some statistics of the runs. The parameters are:
* The solver.
* The number of iterations to evaluate model.
* The arguments, this should be a tuple.

In [10]:
args = (init_path1, 1000.0, 0.01)
statistics = get_stats(TravellingSalesman, 30, args)

In [11]:
pprint(statistics)

{'Best solution': {'f': 248.0, 'x': array([0, 5, 3, 8, 9, 6, 2, 7, 4, 1])},
 'Mean': 261.6666666666667,
 'Standard deviation': 11.169402649898316,
 'Worst solution': {'f': 271.0, 'x': array([0, 5, 3, 8, 9, 6, 2, 7, 1, 4])}}


## 0/1 Knapsack problem

\begin{equation}
  \label{eq:KP}
  \begin{array}{rll}
  \text{maximize:} & f(\vec{x}) = \sum_{i=1}^{n} p_i \cdot x_{i} &  \\
  \text{where: } & g_1(\vec{x}) = \sum_{i=1}^{n} w_i \cdot x_{i}  \leq c &  \\
          &  x_i \in \{0,1\} & i\in\{1,\ldots,n\}\\
  \end{array}
\end{equation}

Consider this input:
- $n = 5$
- $p = \{5, 14, 7, 2, 23\}$
- $w = \{2, 3, 7, 5, 10\}$
- $c = 15$

Where the best solution is:
$x = [1, 1, 0, 0, 1]$ , $f(x) = 42$ and $g_{1}(x) \leq 15$

For more information about this problem check [here](https://en.wikipedia.org/wiki/Knapsack_problem).

The steps to follow are:
- Define our objective function.
- Include the constraints list.
- Define the initial solution.
- Create our solver.
- Optimize the function.


## Objective function
In this case, we use global variables for calculating the objective function and the constraint.

In [12]:
#Define global input.
n = 50
p = [60, 52, 90, 57, 45, 64, 60, 45, 63, 94, 44, 90, 66, 64, 32, 39, 91, 40, 73, 61, 82, 94, 39, 68, 94, 98, 80, 79, 73, 99, 49, 56, 69, 49, 82, 99, 65, 34, 31, 85, 67, 62, 56, 38, 54, 81, 98, 63, 48, 83]
w = [38, 20, 21, 21, 37, 28, 32, 30, 33, 35, 29, 32, 35, 24, 28, 29, 22, 34, 31, 36, 36, 28, 38, 25, 38, 37, 20, 23, 39, 31, 27, 20, 38, 38, 36, 28, 39, 22, 23, 22, 21, 24, 23, 33, 31, 30, 32, 30, 22, 37]
c = 870

In [13]:
@checkargs
def f(x : np.ndarray) -> float:
    global p
    return -1.0* np.dot(x,p)


## Constraints

In [14]:
@checkargs 
def g1(x : np.ndarray) -> bool:
    global w,c
    result = np.dot(x,w)
    g1.__doc__="{} <= {}".format(result,c)
    return result <= c

constraints_list= [g1]

### Initial solution
In this case, we start taking random objects and we try to insert them in the backpack.

In [15]:
def getInitialSolution(NumObjects=15):
    global n,p,w,c
    #Empty backpack
    x = [0 for i in range(n)]
    weight_x = 0
    
    #Random order to insert objects.
    objects = list(range(n))
    np.random.shuffle(objects)
    
    for o in  objects[:NumObjects]:
        #Check the constraint about capacity.
        if weight_x + w[o] <= c:
            x[o] = 1
            weight_x += w[o]
            
    return np.array(x)


## defining solver instance


In [16]:
class Knapsack_solver(SimulatedAnnealing):

    @checkargs
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
    @checkargs
    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        x_ = x.copy()
        N = len(x_)
        while(True):
            ind = random.randint(0, N-1)
            x_[ind] ^= 1
            
            if(self.is_valid(x_)):
                break
            
            x_[ind] ^= 1
        
        return np.array(x_)

In get_neighbor we are explicit about the random solution generated  have to be validated.

In [17]:
Knapsack_ = Knapsack_solver(f, [g1])

In [18]:
Knapsack_.optimize(getInitialSolution,10000.0,0.1)
print(Knapsack_)

Simulated Annealing: 
 f(X) = -2075.0 
 X = [1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0
 1 0 1 1 1 0 0 0 1 0 1 0 0] 
 Constraints: 
 870 <= 870 



## Model confidence 

In [19]:
args = (getInitialSolution,1000.0,0.01)
statistics = get_stats(Knapsack_, 30, args)

In [20]:
pprint(statistics)

{'Best solution': {'f': -2239.0,
                   'x': array([0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1])},
 'Mean': -2128.5666666666666,
 'Standard deviation': 54.19082538175218,
 'Worst solution': {'f': -2040.0,
                    'x': array([0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1])}}


### Improving current model
The current model has huge problems to find the best solution.

In this case we will try to:
* Generate the initial solution in other form.
* Create a new strategy for generate neighbors.


#### Initial solution


In [21]:

def Init_GISP():
    global n,p,w,c
    Arr_=[]
    for i in range(n):
        Arr_.append((p[i]/w[i],i))
    
    Arr_.sort(reverse=True)
    return Arr_
GISP_Arr = Init_GISP()

def getInitialSolution2():
    global n,p,w,c, GISP_Arr
    X = [0 for i in range(n)]
    current_weight = 0
    for i in range(n):
        ind = GISP_Arr[i][1]
        if current_weight+ w[ind] <= c:
            current_weight+=w[ind]
            X[ind] = 1
    return np.array(X)


In [28]:
class Knapsack_solver2(SimulatedAnnealing):

    @checkargs
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
        
    def generate_neighbor(self, x: np.ndarray) -> np.ndarray:
        x_ = x.copy()
        N = len(x_)
        ind = random.randint(0, N-1)
        x_[ind] ^= 1
        
        return x_
    
    def repair_neighbor(self, x: np.ndarray) -> np.ndarray:
        global GISP_Arr,w,c,n
        
        #Get the total weight
        total_weight=0
        for i in range(n):
            if x[i]:
                total_weight += w[i]
        
        for i in range(n):
            ind = GISP_Arr[n - (1+i)][1] #Lowest
            
            if x[ind]:
                total_weight -= w[ind]
                x[ind] = 0
            
            if total_weight <= c:
                break
                
    def improve_neighbor(self, x: np.ndarray) -> np.ndarray:
        global GISP_Arr,w,c,n
        
        #Get the total weight
        total_weight=0
        for i in range(n):
            if x[i]:
                total_weight += w[i]
                
        for i in range(n):
            ind = GISP_Arr[i][1]
            if total_weight + w[ind] > c:
                continue
                
            if x[ind] == 0 :
                total_weight += w[ind]
                x[ind] = 1
            
    @checkargs
    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        neighbor_ = self.generate_neighbor(x)
            
        if(self.is_valid(neighbor_)):
            return neighbor_
        
        #RI strategy
        self.repair_neighbor(neighbor_)
        self.improve_neighbor(neighbor_)
        
        return neighbor_

In [29]:
Knapsack_2 = Knapsack_solver2(f, [g1])

In [54]:
Knapsack_2.optimize(getInitialSolution2,10000.0,0.1)
print(Knapsack_2)

Simulated Annealing: 
 f(X) = -2378.0 
 X = [0 1 1 1 0 1 0 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0
 0 0 1 1 1 1 0 0 1 1 1 1 1] 
 Constraints: 
 865 <= 870 



In [37]:
args = (getInitialSolution2,1000.0,0.1)
statistics = get_stats(Knapsack_, 30, args)

In [38]:
pprint(statistics)

{'Best solution': {'f': -2378.0,
                   'x': array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1])},
 'Mean': -2378.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -2378.0,
                    'x': array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1])}}
